# AGO and AGE Web Map Edit Form Audit

Description: The AGO and AGE Web Map Edit Form Audit will loop through all webmaps in an AGO or AGE organization and return information about any editing forms that are configured in the web maps and any expressions used in the forms. 

Code inspired by: Kelly Wright

Created on: 8/3/25

Purpose: To generate a list of all webmaps that have Edit Forms Configured. If there are any expressions used in the Edit Forms, those are also returned in the list. The list is displayed in the notebook as a dataframe. The dataframe is exported to excel, and saved in the useer defined location in step 3. You can optionally upload the excel file to AGO/AGE as a hosted table. 

Authored By: Joe Guzi

Previous Production Date:       

Production Date: 8/8/25

Tags: AGO, AGE, WebMaps, Editing Forms, Expresssions, Excel, Dataframe

Notes:

# Step 1: Import Libraries

In [ ]:
# import
print("Import Libraries...")
from arcgis.gis import GIS
from arcgis.gis import ItemProperties
from arcgis.gis import ItemTypeEnum
import getpass
from IPython.display import display
import json
import requests
import csv
from tkinter.filedialog import asksaveasfilename
import os
import ipywidgets as widgets
import pandas as pd
pd.options.display.max_rows = None
import urllib3
urllib3.disable_warnings()
print("Import Libraries Complete!")

# Step 2: Connect to ArcGIS Online or ArcGIS Enterprise

Choose an option below to connect to ArcGIS Online or ArcGIS Enterprise

## Connect with current AGO Credentials to this organization

Use this option if you are already signed into AGO and are running this notebook as a hosted notebook in AGO.

In [ ]:
gis = GIS("home")
gis
print("Connect to AGO/AGE Complete!")

## Connect to ArcGIS Online by providing Org URL and Credentials (built-in users)

In [ ]:
# Connection Variables\n",
Organization = input("What's your oranization?")
User = input("What's your  username? ")
Password = getpass.getpass("What's your password?")
# Connection
gis = GIS(Organization, User, Password, verify_cert=False)
print(gis)
print("Connect to AGO/AGE Complete!")

## Connect to ArcGIS Online by providing a profile

We connect by using a profile. This profile has been setup beforehand by using built-in credentials and registering it with the profile itself. We have a section in the documentation that shows you how to do this if you are interested.

In [ ]:
# Connection Variables
gis = GIS(profile="your_online_profile") # edit "your_online_profile" with the name of your profile
gis
print("Connect to AGO/AGE Complete!")

## User authentication with OAuth 2.0

https://developers.arcgis.com/python/guide/working-with-different-authentication-schemes/#user-authentication-with-oauth-20

In [ ]:
# Connection Variables
Organization = input("What's your oranization? ")
OrgClientID = input("What's your  ClientID? ")
gis = GIS(Organization, client_id=OrgClientID)
print("Successfully logged in as: " + gis.properties.user.username)
print("Connect to AGO/AGE Complete!")"

## Connect to ArcGIS Enterprise with Single Sign On

In [ ]:
# Connection Variables
Organization = input("What's your oranization? ")
gis = GIS(Organization)
gis
print("Logged in as " + gis.properties.user.username + " to " + gis.properties.portalName)
print("Connect to AGO/AGE Complete!")

# Step 3: Audit Setup
Run the following sections to set up the Audit process. If a section is marked optional, you do not have to run that section.

## Functions

In [ ]:
print("Begin Function Section.")
# This is where you define Functions

# Begin Check Dictionary Key Function
def CheckDictionaryKey(Dictionary, Key):
    value = Dictionary.get(Key)
    #print(value)
    if value == -1:
        #print("negative1")
        return 0
    elif value is not None:
        return(value)
    else:
        return "none"
# End Check Dictionary Key Function

# Begin Check Expression Infor Audit Function
def CheckExpressionInfoAudit(EditFormAudit, ValueExpressionName):
    if ValueExpressionName != "none":
        ExpressionInfo = CheckDictionaryKey(EditFormAudit,"expressionInfos")
        if ExpressionInfo != "none":
            #print(ExpressionInfo)
            for Expression in ExpressionInfo:
                #print(Expression)
                ExpressionName =  CheckDictionaryKey(Expression,"name")
                if ValueExpressionName == ExpressionName:
                    ExpressionNameValue = CheckDictionaryKey(Expression,"name")
                    ExpressionTitleValue = CheckDictionaryKey(Expression,"title")
                    ExpressionReturnTypeValue = CheckDictionaryKey(Expression,"returnType")
                    ExpressionValue = CheckDictionaryKey(Expression,"expression")
                    return ExpressionNameValue, ExpressionTitleValue, ExpressionReturnTypeValue, ExpressionValue
                else:
                    # return something
                    Message = "Not the correct expression. Try Again."
                    #print(Message)
        else:
            #print("There is no Expression")
            # return something
            ExpressionNameValue = "No Expression"
            ExpressionTitleValue = ""
            ExpressionReturnTypeValue = ""
            ExpressionValue = ""
            return ExpressionNameValue, ExpressionTitleValue, ExpressionReturnTypeValue, ExpressionValue
    else:
            #print("There is no Expression")
            # return something
            ExpressionNameValue = "No Expression"
            ExpressionTitleValue = ""
            ExpressionReturnTypeValue = ""
            ExpressionValue = ""
            return ExpressionNameValue, ExpressionTitleValue, ExpressionReturnTypeValue, ExpressionValue
# End Check Expression Infor Audit Function

print("Function Section Complete!")

## Create a folder to save the reports

In [ ]:
filepath = asksaveasfilename(title="Select a location to create a folder")
print(filepath)
os.makedirs(filepath, exist_ok=True) # Creates folder and parent folders if needed

# Step 4: Execute Code Section

In [ ]:
print("Begin Execute Code Section.") 
# this is where you put the code you want to execute.

Audit = gis.content.search(query="", item_type="Web Map", max_items = 10000)
#Audit = gis.content.search(query="id:InsertItemIdForTesting", item_type="Web Map", max_items = 10000) # Item Id of a Web Map

WebMapEditFormAuditList = []
    
for Item in Audit:
    ItemTitle = str(Item["title"].replace(',', ""))
    ItemID = str(Item["id"])
    print(ItemID)
    ItemType = str(Item["type"])
    ItemOwner = str(Item["owner"])
    ItemAccess = str(Item["access"])
    ItemContentStatus = str(Item.get("contentStatus"))
    ItemData = Item.get_data(try_json=True)
    for ItemInfo in ItemData:
        # Layers
        if ItemInfo == "operationalLayers":
            layers = ItemData["operationalLayers"]
            for layer in layers:
                #print (layer)
                EditForm = CheckDictionaryKey(layer, "formInfo")
                #print(EditForm)
                if EditForm != "none":
                    try:
                        FormElements = EditForm["formElements"][0]["formElements"]
                    except:
                        FormElements = EditForm["formElements"]
                    #print(FormElements)
                    for FormElement in FormElements:
                        #print(FormElement)
                        WebMapEditFormAuditDictionary = {}
                        WebMapEditFormAuditDictionary["ItemTitle"] = ItemTitle.replace(",", "")
                        WebMapEditFormAuditDictionary["ItemID"] = ItemID
                        WebMapEditFormAuditDictionary["ItemType"] = ItemType
                        WebMapEditFormAuditDictionary["ItemOwner"] = ItemOwner
                        WebMapEditFormAuditDictionary["ItemAccess"] = ItemAccess
                        WebMapEditFormAuditDictionary["ItemContentStatus"] = ItemContentStatus
                        WebMapEditFormAuditDictionary["LayerID"] = CheckDictionaryKey(layer,"id")
                        WebMapEditFormAuditDictionary["LayerTitle"] = CheckDictionaryKey(layer,"title")
                        WebMapEditFormAuditDictionary["LayerType"] = CheckDictionaryKey(layer,"layerType")
                        WebMapEditFormAuditDictionary["LayerVisibility"] = CheckDictionaryKey(layer,"visibility")
                        WebMapEditFormAuditDictionary["LayerURL"] = CheckDictionaryKey(layer,"url")
                        WebMapEditFormAuditDictionary["LayerItemID"] = CheckDictionaryKey(layer,"itemId")
                        WebMapEditFormAuditDictionary["ContainsEditForm"] = "Yes"
                        WebMapEditFormAuditDictionary["FormfieldName"] = CheckDictionaryKey(FormElement,"fieldName")
                        WebMapEditFormAuditDictionary["FormfieldLabel"] = CheckDictionaryKey(FormElement,"label")
                        WebMapEditFormAuditDictionary["FormfieldEditable"] = CheckDictionaryKey(FormElement,"editableExpression")
                        WebMapEditFormAuditDictionary["FormfieldEVisisble"] = CheckDictionaryKey(FormElement,"visibilityExpression")
                        WebMapEditFormAuditDictionary["FormfieldInputType"] = CheckDictionaryKey(FormElement,"inputType")
                        ValueExpression = CheckDictionaryKey(FormElement, "valueExpression")
                        #print(ValueExpression)
                        # Call Check Expression Info Audit Function
                        ExpressionName, ExpressionTitle, ExpressionReturnType, Expression = CheckExpressionInfoAudit(EditForm, ValueExpression)
                        # Add Values returned from Check Expression Info Audit Function to the WebMapEditFormAuditDictionary
                        WebMapEditFormAuditDictionary["ExpressionName"] = ExpressionName
                        WebMapEditFormAuditDictionary["ExpressionTitle"] = ExpressionTitle
                        WebMapEditFormAuditDictionary["ExpressionReturnType"] = ExpressionReturnType
                        WebMapEditFormAuditDictionary["Expression"] = Expression
                        WebMapEditFormAuditList.append(WebMapEditFormAuditDictionary)

#add  dataframe code here
#print(WebMapEditFormAuditList)
df = pd.DataFrame(WebMapEditFormAuditList)
display(df)
#export dataframe to excel
ExcelFile = filepath + "/" + "WebMapFormAudit.xlsx"
print(ExcelFile)
df.to_excel(ExcelFile, index=False)
print("Execute Code Section Complete!") 

# Step 5: Upload to AGO (Optional)

In [ ]:
print("Begin Upload to AGO Section.") 
file = ExcelFile.rpartition('/')
#print(file)
filename = file[2]
#print(file[2])
testfile = gis.content.search(query="title:"+filename, item_type="Feature Layer Collection")
if testfile:
    print("Found the service")
    testfile = testfile[0]
    display(testfile)
    from arcgis.features import FeatureLayerCollection
    testfile_collection = FeatureLayerCollection.fromitem(testfile)
    print("The table already exists")
    testfile_collection.manager.overwrite(ExcelFile)
    display
else:
    print("We have to upload it")
    print("Define User")
    User = gis.users.me.username
    Folder = input("What folder would you like to create your item?")
    # Check if Folder Exists
    FolderExists = gis.content.folders.get(folder=Folder, owner=User)
    print(FolderExists)
    if FolderExists == None:
        print("Create Folder")
        gis.content.folders.create(Folder)
    ItemFolder = gis.content.folders.get(folder=Folder, owner=User)
    print("Define Item Properties")
    #ItemTypeEnum.Microsoft-Excel.value
    item_props = ItemProperties(title=filename, item_type= 'Microsoft Excel')
    print("Add Excel to AGO")
    csv_future_item = ItemFolder.add(item_properties=item_props, file=ExcelFile)
    csv_item = csv_future_item.result()
    display(csv_item)
    print("Publish Excel as Table Layer")
    csv_lyr = csv_item.publish()
    display(csv_lyr)
    
print ("The Excel has been published.\nThe Script is Complete!")